In [1]:
#Imports
import os
import requests
import arcpy
import subprocess
from zipfile import ZipFile

#Initial setup
project = arcpy.mp.ArcGISProject("CURRENT")
map = project.listMaps()[0]
ref = arcpy.SpatialReference(26915)  #NAD zone 15N

surfaceUrl = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/wabasha/laz/' #Elevation data download link
landcoverUrl = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2016/tif_biota_landcover_nlcd_mn_2016.zip' #Land cover data download link

outputPath = r"C:\ArcGIS\Projects\Lab2-2" #Where everything is saved
if not os.path.exists(outputPath): #Test that outputPath exists
    os.makedirs(outputPath)
    print("Created output directory: ", outputPath)
else:
    print("Output directory is already assigned to: ", outputPath)

Output directory is already assigned to:  C:\ArcGIS\Projects\Lab2-2


In [2]:
#LAZ files

#Variable names
dnrFiles = [ #So named to differentiate from lasFiles
    '4342-28-61.laz', '4342-28-60.laz',
    '4342-29-60.laz', '4342-29-61.laz'
]
laszipUrl = surfaceUrl + 'laszip.exe'
laszipPath = os.path.join(outputPath, 'laszip.exe')

for dnrFile in dnrFiles:
    dnrPath = os.path.join(outputPath, dnrFile)
    if not os.path.exists(dnrPath):
        response = requests.get(surfaceUrl + dnrFile, stream=True)
        if response.status_code == 200:
            with open(dnrPath, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded {dnrFile}")
        else:
            print(f"Failed to download {dnrFile}, status code: {response.status_code}")
    else:
        print(f"{dnrFile} already exists at {dnrPath}")

#Download laszip.exe if not already done
if not os.path.exists(laszipPath):
    response = requests.get(laszipUrl)
    if response.status_code == 200:
        with open(laszipPath, 'wb') as file:
            file.write(response.content)
        print("Downloaded laszip.exe")
    else:
        print(f"Failed to download laszip.exe, status code: {response.status_code}")
else:
    print("laszip.exe already downloaded.")

#Ensure laszip.exe is executable
if os.name == 'nt':
    os.chmod(laszipPath, 0o755)

4342-28-61.laz already exists at C:\ArcGIS\Projects\Lab2-2\4342-28-61.laz
4342-28-60.laz already exists at C:\ArcGIS\Projects\Lab2-2\4342-28-60.laz
4342-29-60.laz already exists at C:\ArcGIS\Projects\Lab2-2\4342-29-60.laz
4342-29-61.laz already exists at C:\ArcGIS\Projects\Lab2-2\4342-29-61.laz
laszip.exe already downloaded.


In [3]:
#LAS files & LASD DEM
lasFiles = [os.path.join(outputPath, file.replace('.laz', '.las')) for file in dnrFiles]
lasdPath = os.path.join(outputPath, 'elevation.lasd')
demPath = os.path.join(outputPath, 'elevation_DEM')

#Unzip .laz files to .las files using laszip
def laszip(dnrPath, lasPath):
    try:
        # Run the laszip command
        result = subprocess.run(
            [laszipPath, '-i', dnrPath, '-o', lasPath],
            capture_output=True, text=True
        )

        # Check for success or handle warnings/errors
        if result.returncode == 0:
            print(f"Successfully unzipped {dnrPath} to {lasPath}")
        else:
            print(f"Failed to unzip {dnrPath}: {result.stderr}")
            print(f"Command output: {result.stdout}")

    except FileNotFoundError:
        print(f"laszip.exe not found at {laszipPath}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Check if .las files already exist, otherwise unzip
for dnrFile in dnrFiles:
    dnrPath = os.path.join(outputPath, dnrFile)
    lasPath = os.path.join(outputPath, dnrFile.replace('.laz', '.las'))
    
    if not os.path.exists(lasPath):
        if os.path.exists(dnrPath):
            lazip(dnrPath, lasPath)
        else:
            print(f"{dnrFile} not found.")
    else:
        print(f"{dnrFile.replace('.laz', '.las')} already exists.")

4342-28-61.las already exists.
4342-28-60.las already exists.
4342-29-60.las already exists.
4342-29-61.las already exists.


In [4]:
#Create LAS Dataset if not already created
if not os.path.exists(lasdPath):
    arcpy.management.CreateLasDataset(lasFiles, lasdPath)
    print(f"Created LAS dataset: {lasdPath}")
else:
    print(f"LAS dataset already exists at: {lasdPath}")

# Convert LASD to DEM
if not os.path.exists(demPath):
    arcpy.conversion.LasDatasetToRaster(
        lasdPath, demPath,
        value_field="ELEVATION",
        interpolation_type="BINNING AVERAGE NATURAL_NEIGHBOR",
        data_type="FLOAT",
        sampling_type="CELLSIZE",
        sampling_value=2
    )
    print(f"Created DEM: {demPath}")
else:
    print(f"DEM already exists at: {demPath}")

LAS dataset already exists at: C:\ArcGIS\Projects\Lab2-2\elevation.lasd
DEM already exists at: C:\ArcGIS\Projects\Lab2-2\elevation_DEM


In [5]:
#Land cover raster

#Variable names
zipPath = os.path.join(outputPath, 'landcover.zip')
tifPath = os.path.join(outputPath, 'NLCD_2016_Land_Cover.tif')
clipPath = os.path.join(outputPath, 'land_cover_clip.tif')

#Download land cover raster
if not os.path.exists(zipPath):
    response = requests.get(landcoverUrl)
    if response.status_code == 200:
        with open(zipPath, 'wb') as file:
            file.write(response.content)
        print("Downloaded land cover ZIP file.")
    else:
        print("Failed to download land cover ZIP file, status code: ", response.status_code)
else:
    print("Land cover ZIP file already exists.")

#Extract the ZIP file
if not os.path.exists(os.path.join(outputPath, 'NLCD_2016_Land_Cover.tif')):
    with ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall(outputPath)
        print("Extracted all files from the ZIP archive.")

#Check if the raster file exists before trying to add it
if os.path.isfile(tifPath):
    # Add the raster to the map
    map.addDataFromPath(tifPath)
    print(f"Added {tifPath} to the map.")
else:
    print("Error: The file NLCD_2016_Land_Cover.tif was not found after extraction.")
    
#Clip the new raster to the extent of the DEM
arcpy.management.Clip(
    in_raster="NLCD_2016_Land_Cover.tif",
    out_raster=r"C:\ArcGIS\Projects\Lab2-2\land_cover_clip.tif",
    in_template_dataset="elevation_DEM",  #Set the extent based on the DEM layer
    nodata_value="255",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

#Check that clip worked and delete large .tif file
for layer in map.listLayers():
    if layer.name == "land_cover_clip.tif":
        print("Layer land_cover_clip.tif added to the map.")
    elif layer.name == "NLCD_2016_Land_Cover.tif":
        map.removeLayer(layer)
        print("Layer NLCD_2016_Land_Cover.tif removed from the map.")

Land cover ZIP file already exists.
Added C:\ArcGIS\Projects\Lab2-2\NLCD_2016_Land_Cover.tif to the map.
Layer land_cover_clip.tif added to the map.
Layer NLCD_2016_Land_Cover.tif removed from the map.


In [6]:
#Create slope raster for cost surface
slopePath = os.path.join(outputPath, 'slope_DEM.tif')

with arcpy.EnvManager(scratchWorkspace=outputPath):
    slope_DEM = arcpy.sa.Slope(
        in_raster="elevation_DEM",
        output_measurement="DEGREE",
        z_factor=1,
        method="PLANAR",
        z_unit="METER",
        analysis_target_device="GPU_THEN_CPU"
    )
    slope_DEM.save(slopePath)

if arcpy.Exists(slopePath):
    print(f"Slope raster successfully saved as: {slopePath}")
else:
    print("Failed to save the slope raster.")

Slope raster successfully saved as: C:\ArcGIS\Projects\Lab2-2\slope_DEM.tif


In [10]:
#Make raster where water & fields are weighted
land_cover_weight = arcpy.sa.Con(
    (arcpy.sa.Raster(clipPath) == 11) | #Water
    (arcpy.sa.Raster(clipPath) == 81) | #Pasture
    (arcpy.sa.Raster(clipPath) == 82),  #Crops
    10, 
    1
)

weightPath = os.path.join(outputPath, 'land_cover_weight.tif')
land_cover_weight.save(weightPath)

print(f"land_cover_weight saved at: {weightPath}")

land_cover_weight saved at: C:\ArcGIS\Projects\Lab2-2\land_cover_weight.tif


In [9]:
#Make the cost surface
costSurface = (arcpy.sa.Raster(slopePath) + arcpy.sa.Raster(weightPath)) / 2

costSurfacePath = os.path.join(outputPath, 'cost_surface.tif')
costSurface.save(costSurfacePath)

print(f"Cost surface saved at: {costSurfacePath}")

Cost surface saved at: C:\ArcGIS\Projects\Lab2-2\cost_surface.tif


In [8]:
#Define start and end points
start = arcpy.Point(568098, 4886440) #easting, northing
startPath = os.path.join(outputPath, 'start.shp')
end = arcpy.Point(570377, 4882902) #Picked a random point in the Whitewater WMA but not in water
endPath = os.path.join(outputPath, 'end.shp')

#Create the start point feature class if it does not exist
if not arcpy.Exists(startPath):
    arcpy.management.CreateFeatureclass(outputPath, 'start.shp', 'POINT', spatial_reference=ref)
    with arcpy.da.InsertCursor(startPath, ['SHAPE@']) as cursor:
        cursor.insertRow([arcpy.PointGeometry(start, ref)])
    print("Start point feature class created and populated.")
else:
    print("Start point feature class already exists.")

#Create the end point feature class if it does not exist
if not arcpy.Exists(endPath):
    arcpy.management.CreateFeatureclass(outputPath, 'end.shp', 'POINT', spatial_reference=ref)
    with arcpy.da.InsertCursor(endPath, ['SHAPE@']) as cursor:
        cursor.insertRow([arcpy.PointGeometry(end, ref)])
    print("End point feature class created and populated.")
else:
    print("End point feature class already exists.")

Start point feature class already exists.
End point feature class already exists.


In [15]:
#Paths to necessary files
distancePath = os.path.join(outputPath, 'cost_distance.tif')
backlinkPath = os.path.join(outputPath, 'backlink.tif')
optimalPath = os.path.join(outputPath, 'optimal_path.tif')

#Distance accumulation Raster
out_distance_accumulation = arcpy.sa.DistanceAccumulation(
    in_source_data=startPath,
    in_barrier_data=None,
    in_surface_raster=slopePath,  #Use slope_DEM raster
    in_cost_raster=costSurfacePath,  #Use cost_surface raster
    in_vertical_raster=None,
    vertical_factor="BINARY 1 -30 30",
    in_horizontal_raster=None,
    horizontal_factor="BINARY 1 45",
    out_back_direction_raster=backlinkPath,  #Save backlink raster
    out_source_direction_raster=None,
    out_source_location_raster=None,
    source_initial_accumulation=None,
    source_maximum_accumulation=None,
    source_cost_multiplier=None,
    source_direction="",
    distance_method="PLANAR"
)

out_distance_accumulation.save(distancePath)
print(f"Distance Accumulation raster saved at: {distancePath}")
print(f"Backlink raster saved at: {backlinkPath}")

with arcpy.EnvManager(scratchWorkspace=outputPath):
    out_path_accumulation_raster = arcpy.sa.OptimalPathAsRaster(
        in_destination_data="end",
        in_distance_accumulation_raster="cost_distance",
        in_back_direction_raster=r"C:\ArcGIS\Projects\Lab2-2\backlink.tif",
        destination_field="Id",
        path_type="BEST_SINGLE"
    )
    
out_path_accumulation_raster.save(optimalPath)
print(f"Optimal Path raster saved at: {optimalPath}")

Distance Accumulation raster saved at: C:\ArcGIS\Projects\Lab2-2\cost_distance.tif
Backlink raster saved at: C:\ArcGIS\Projects\Lab2-2\backlink.tif
Optimal Path raster saved at: C:\ArcGIS\Projects\Lab2-2\optimal_path.tif
